[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/MasahiroAraki/masahiroaraki.github.io/blob/master/MMI-meeting/chap04.ipynb)

## chap04

1.   データが置かれたGoogleドライブをマウント（Colabのメニューで操作）


> ファイル -> ドライブをマウント -> 実行可能なセルが現れる -> ログインして認証コードを入力




2.   /content/driveにマウントされる



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


GiNZAのインストール  

In [2]:
!pip install -U ginza
import pkg_resources, imp
imp.reload(pkg_resources)

     |████████████████████████████████| 54.9MB 70kB/s 
     |████████████████████████████████| 81kB 10.5MB/s 
     |████████████████████████████████| 44.8MB 92kB/s 
     |████████████████████████████████| 481kB 47.0MB/s 
  Created wheel for ginza: filename=ginza-3.1.2-cp36-none-any.whl size=17311 sha256=37ea7dfc3137164e0313cef865ffa98292da2b57431527cfa1d4c2c1323149ad
  Stored in directory: /root/.cache/pip/wheels/70/8d/57/f089078acc0dbaebffc08c178e9f20924fa794c114ad36f7f7
  Created wheel for ja-ginza: filename=ja_ginza-3.1.0-cp36-none-any.whl size=54963619 sha256=af8d93e05da8567b16a2ece1f079e0f0c9398af64a2c004b5a51e65bb9e0f757
  Stored in directory: /root/.cache/pip/wheels/f2/8a/07/1837eeb5c5648fa8d266102b78a894e495234585ac3f024cf1
  Created wheel for ja-ginza-dict: filename=ja_ginza_dict-3.1.0-cp36-none-any.whl size=70877544 sha256=478ae7928b2f5391b8cbde272d8594ba30e57e56297b6a67459a98586cfd142c
  Stored in directory: /root/.cache/pip/wheels/30/88/d7/7f0692ba26060966af34538e1079438d16

<module 'pkg_resources' from '/usr/local/lib/python3.6/dist-packages/pkg_resources/__init__.py'>

GiNZAの使用準備

In [0]:
import spacy
nlp = spacy.load('ja_ginza')

京都大学情報学研究科--NTTコミュニケーション科学基礎研究所 共同研究ユニット 
 [解析済みブログコーパス](http://nlp.ist.i.kyoto-u.ac.jp/kuntt/#ga739fe2)
の読み込み

In [0]:
Dpath = '/content/drive/My Drive/Resources/KNBC_v1.0_090925_utf8/corpus2/'
topics = ['Gourmet', 'Keitai', 'Kyoto', 'Sports']

In [0]:
import pandas as pd
import re

cols = ['doc', 'label']
df = pd.DataFrame(index=[], columns=cols)
p = re.compile(r'(［.*］)(.*)')

for topic in topics:
  data = pd.read_table(Dpath+topic+'.tsv', header=None, dtype=str, usecols=[1])
  for d in data.itertuples():
    m = p.match(d[1])
    if m != None:  # first line of the document
      if d[0] != 0:
        df = df.append(pd.DataFrame([doc.strip(), topic], index = cols).T)
      sent = m.groups()[1]
      doc = ''
    else:
      sent = d[1]
    doc = doc + ' ' +  ' '.join([t.text for t in nlp(sent)])
  df = df.append(pd.DataFrame([doc.strip(), topic], index = cols).T)

In [6]:
df

,doc,label
0,烏丸 六角 の おかき 屋 さん 六角堂 の 前 に ある 、 蕪村 庵 と いう お店 に...,Gourmet
0,河原町 の 居酒屋 この 間 先輩 たち に つれ られ 、 河原町 の 居酒屋 へ 行っ ...,Gourmet
0,ちょっと 贅沢 ほんの ちょこっと な ん だ けど 、 贅沢 し たい とき に 何 を ...,Gourmet
0,食欲 の 秋 最近 おなか が 減っ て 仕方 あり ませ ん か 。 仕方 あり ませ ん...,Gourmet
0,季節 限定 も のっ て … １０ 月 と いえ ば 秋 本番 で ある 。 栗 や 秋刀魚...,Gourmet
...,...,...
0,クライマックス シリーズ 今年 の 阪神 は なん やっ た ん や ！ ！ 初め 弱く て...,Sports
0,京都 サンガ に つい て この ブログ の 右側 に 検索 窓 ある じゃ ない です か...,Sports
0,サッカー から 伺える ルール と の 付き合い 方 私自身 は 全く の サッカー 素人 ...,Sports
0,する か 見る か スポーツ は する 派 です か ？ し たい 見る 派 です か ？ ...,Sports


In [7]:
df['label'].value_counts(sort=False)

Gourmet    57
Kyoto      91
Keitai     79
Sports     22
Name: label, dtype: int64

# 文書分類

In [0]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import classification_report

In [0]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(df.values[:,1])
X = df.values[:,0]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

## Naive Bayes

特徴ベクトルは単語数え上げ

In [11]:
clf1 = Pipeline([('vect', CountVectorizer()), ('mnb', MultinomialNB())])
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
print(classification_report(y_test, y_pred, target_names=topics))

              precision    recall  f1-score   support

     Gourmet       1.00      0.74      0.85        19
      Keitai       0.78      1.00      0.88        21
       Kyoto       0.81      1.00      0.90        26
      Sports       1.00      0.22      0.36         9

    accuracy                           0.84        75
   macro avg       0.90      0.74      0.75        75
weighted avg       0.87      0.84      0.81        75



## ロジステック回帰

特徴ベクトルはTfIdf

In [12]:
clf2 = Pipeline([('vect', TfidfVectorizer()), ('lr', LogisticRegression())])
clf2.fit(X_train, y_train)
y_pred = clf2.predict(X_test)
print(classification_report(y_test, y_pred, target_names=topics))

              precision    recall  f1-score   support

     Gourmet       0.92      0.63      0.75        19
      Keitai       0.95      0.90      0.93        21
       Kyoto       0.60      0.96      0.74        26
      Sports       0.00      0.00      0.00         9

    accuracy                           0.75        75
   macro avg       0.62      0.62      0.60        75
weighted avg       0.71      0.75      0.70        75



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## オンラインSVM

特徴ベクトルはTfIdf

In [13]:
clf3 = Pipeline([('vect', TfidfVectorizer()), ('sgd', SGDClassifier())])
clf3.fit(X_train, y_train)
y_pred = clf3.predict(X_test)
print(classification_report(y_test, y_pred, target_names=topics))

              precision    recall  f1-score   support

     Gourmet       0.95      0.95      0.95        19
      Keitai       0.95      0.90      0.93        21
       Kyoto       0.80      0.92      0.86        26
      Sports       0.83      0.56      0.67         9

    accuracy                           0.88        75
   macro avg       0.88      0.83      0.85        75
weighted avg       0.88      0.88      0.88        75

